# Implement Continious Bag of Words Model (CBOW)

# Imports

In [1]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
import numpy as np
import keras.backend as K
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Lambda, Embedding
from tensorflow.keras.utils import to_categorical

# Load Data

In [3]:
file = open("Zarathusta.txt", "r")
data = file.read()
print(data)

 When Zarathustra was thirty years old, he left his home and the lake of his home, and went into the mountains. There he enjoyed his spirit and solitude, and for ten years did not weary of it. But at last his heart changed,—and rising one morning with the rosy dawn, he went before the sun, and spake thus unto it:

Thou great star! What would be thy happiness if thou hadst not those for whom thou shinest!

For ten years hast thou climbed hither unto my cave: thou wouldst have wearied of thy light and of the journey, had it not been for me, mine eagle, and my serpent.

But we awaited thee every morning, took from thee thine overflow and blessed thee for it.

Lo! I am weary of my wisdom, like the bee that hath gathered too much honey; I need hands outstretched to take it.

I would fain bestow and distribute, until the wise have once more become joyous in their folly, and the poor happy in their riches.

Therefore must I descend into the deep: as thou doest in the evening, when thou goest 

In [4]:
words = data.split()
words

['When',
 'Zarathustra',
 'was',
 'thirty',
 'years',
 'old,',
 'he',
 'left',
 'his',
 'home',
 'and',
 'the',
 'lake',
 'of',
 'his',
 'home,',
 'and',
 'went',
 'into',
 'the',
 'mountains.',
 'There',
 'he',
 'enjoyed',
 'his',
 'spirit',
 'and',
 'solitude,',
 'and',
 'for',
 'ten',
 'years',
 'did',
 'not',
 'weary',
 'of',
 'it.',
 'But',
 'at',
 'last',
 'his',
 'heart',
 'changed,—and',
 'rising',
 'one',
 'morning',
 'with',
 'the',
 'rosy',
 'dawn,',
 'he',
 'went',
 'before',
 'the',
 'sun,',
 'and',
 'spake',
 'thus',
 'unto',
 'it:',
 'Thou',
 'great',
 'star!',
 'What',
 'would',
 'be',
 'thy',
 'happiness',
 'if',
 'thou',
 'hadst',
 'not',
 'those',
 'for',
 'whom',
 'thou',
 'shinest!',
 'For',
 'ten',
 'years',
 'hast',
 'thou',
 'climbed',
 'hither',
 'unto',
 'my',
 'cave:',
 'thou',
 'wouldst',
 'have',
 'wearied',
 'of',
 'thy',
 'light',
 'and',
 'of',
 'the',
 'journey,',
 'had',
 'it',
 'not',
 'been',
 'for',
 'me,',
 'mine',
 'eagle,',
 'and',
 'my',
 'serpe

In [5]:
words = [w.lower().replace("!","").replace(",","").replace(".","").replace("\n", "") for w in words]

In [6]:
words

['when',
 'zarathustra',
 'was',
 'thirty',
 'years',
 'old',
 'he',
 'left',
 'his',
 'home',
 'and',
 'the',
 'lake',
 'of',
 'his',
 'home',
 'and',
 'went',
 'into',
 'the',
 'mountains',
 'there',
 'he',
 'enjoyed',
 'his',
 'spirit',
 'and',
 'solitude',
 'and',
 'for',
 'ten',
 'years',
 'did',
 'not',
 'weary',
 'of',
 'it',
 'but',
 'at',
 'last',
 'his',
 'heart',
 'changed—and',
 'rising',
 'one',
 'morning',
 'with',
 'the',
 'rosy',
 'dawn',
 'he',
 'went',
 'before',
 'the',
 'sun',
 'and',
 'spake',
 'thus',
 'unto',
 'it:',
 'thou',
 'great',
 'star',
 'what',
 'would',
 'be',
 'thy',
 'happiness',
 'if',
 'thou',
 'hadst',
 'not',
 'those',
 'for',
 'whom',
 'thou',
 'shinest',
 'for',
 'ten',
 'years',
 'hast',
 'thou',
 'climbed',
 'hither',
 'unto',
 'my',
 'cave:',
 'thou',
 'wouldst',
 'have',
 'wearied',
 'of',
 'thy',
 'light',
 'and',
 'of',
 'the',
 'journey',
 'had',
 'it',
 'not',
 'been',
 'for',
 'me',
 'mine',
 'eagle',
 'and',
 'my',
 'serpent',
 'but',


In [7]:
vocab = set(words)
vocab

{'a',
 'about',
 'again',
 'also',
 'am',
 'and',
 'as',
 'at',
 'awaited',
 'be',
 'become',
 'bee',
 'been',
 'before',
 'began',
 'behind',
 'behold',
 'bestow',
 'bless',
 'blessed',
 'bliss',
 'but',
 'canst',
 'carry',
 'cave:',
 'changed—and',
 'climbed',
 'cup',
 'dawn',
 'deep:',
 'descend',
 'did',
 'distribute',
 'doest',
 'down',
 'down-going',
 'eagle',
 'empty',
 'enjoyed',
 'envy',
 'even',
 'evening',
 'every',
 'everywhere',
 'exuberant',
 'eye',
 'fain',
 'flow',
 'folly',
 'for',
 'from',
 'gathered',
 'givest',
 'go',
 'goest',
 'going',
 'golden',
 'great',
 'greatest',
 'had',
 'hadst',
 'hands',
 'happiness',
 'happy',
 'hast',
 'hath',
 'have',
 'he',
 'heart',
 'his',
 'hither',
 'home',
 'honey;',
 'i',
 'if',
 'in',
 'into',
 'is',
 'it',
 'it:',
 'itself',
 'journey',
 'joyous',
 'lake',
 'last',
 'left',
 'light',
 'like',
 'lo',
 'man',
 'may',
 'me',
 'men',
 'mine',
 'more',
 'morning',
 'mountains',
 'much',
 'must',
 'my',
 'need',
 'nether-world',
 'n

In [8]:
vocab_size = len(vocab)

In [9]:
vocab_size

165

In [10]:
words_to_idx = {word:i for i,word in enumerate(vocab)}
idx_to_words = {i:word for i,word in enumerate(vocab)}

In [11]:
words_to_idx

{'and': 0,
 'spirit': 1,
 'bless': 2,
 'sea': 3,
 'sun': 4,
 'man': 5,
 'folly': 6,
 'without': 7,
 'for': 8,
 'solitude': 9,
 'with': 10,
 'happiness': 11,
 'wise': 12,
 'behind': 13,
 'last': 14,
 'need': 15,
 'exuberant': 16,
 'weary': 17,
 'serpent': 18,
 'golden': 19,
 'awaited': 20,
 'too': 21,
 'nether-world': 22,
 'enjoyed': 23,
 'climbed': 24,
 'from': 25,
 'wearied': 26,
 'bestow': 27,
 'he': 28,
 'poor': 29,
 'until': 30,
 'wouldst': 31,
 'dawn': 32,
 'my': 33,
 'into': 34,
 'take': 35,
 'bee': 36,
 'shinest': 37,
 'if': 38,
 'me': 39,
 'great': 40,
 'out': 41,
 'mine': 42,
 'when': 43,
 'years': 44,
 'go': 45,
 'going': 46,
 'eagle': 47,
 'wisdom': 48,
 'then': 49,
 'light': 50,
 'zarathustra’s': 51,
 'cup': 52,
 'lo': 53,
 'a': 54,
 'that': 55,
 'in': 56,
 'everywhere': 57,
 'men': 58,
 'therefore': 59,
 'say': 60,
 'every': 61,
 'gathered': 62,
 'empty': 63,
 'to': 64,
 'his': 65,
 'unto': 66,
 'at': 67,
 'overflow': 68,
 'blessed': 69,
 'canst': 70,
 'rising': 71,
 'hone

In [12]:
idx_to_words

{0: 'and',
 1: 'spirit',
 2: 'bless',
 3: 'sea',
 4: 'sun',
 5: 'man',
 6: 'folly',
 7: 'without',
 8: 'for',
 9: 'solitude',
 10: 'with',
 11: 'happiness',
 12: 'wise',
 13: 'behind',
 14: 'last',
 15: 'need',
 16: 'exuberant',
 17: 'weary',
 18: 'serpent',
 19: 'golden',
 20: 'awaited',
 21: 'too',
 22: 'nether-world',
 23: 'enjoyed',
 24: 'climbed',
 25: 'from',
 26: 'wearied',
 27: 'bestow',
 28: 'he',
 29: 'poor',
 30: 'until',
 31: 'wouldst',
 32: 'dawn',
 33: 'my',
 34: 'into',
 35: 'take',
 36: 'bee',
 37: 'shinest',
 38: 'if',
 39: 'me',
 40: 'great',
 41: 'out',
 42: 'mine',
 43: 'when',
 44: 'years',
 45: 'go',
 46: 'going',
 47: 'eagle',
 48: 'wisdom',
 49: 'then',
 50: 'light',
 51: 'zarathustra’s',
 52: 'cup',
 53: 'lo',
 54: 'a',
 55: 'that',
 56: 'in',
 57: 'everywhere',
 58: 'men',
 59: 'therefore',
 60: 'say',
 61: 'every',
 62: 'gathered',
 63: 'empty',
 64: 'to',
 65: 'his',
 66: 'unto',
 67: 'at',
 68: 'overflow',
 69: 'blessed',
 70: 'canst',
 71: 'rising',
 72: '

# Generate Context-Target Pairs

In [13]:
c = 2
dim = 100

In [14]:
X_train = []
Y_train = []

for index, i in enumerate(words):
    
    if(index >= len(words)-2) or (index <= 1):
        continue
    start = index -c
    end = index + c + 1
    
    context = words[start:end]
    context.remove(i)
    target = i
    
    X_train.append([words_to_idx[w] for w in context])
    Y_train.append([words_to_idx[target]])
    
    

In [15]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_train


array([[ 43,  80, 141,  44],
       [ 80, 118,  44,  96],
       [118, 141,  96,  28],
       ...,
       [144,  54,  89, 122],
       [ 54,   5, 122,  51],
       [  5,  89,  51,  74]])

In [16]:
Y_train

array([[118],
       [141],
       [ 44],
       [ 96],
       [ 28],
       [ 76],
       [ 65],
       [ 97],
       [  0],
       [ 98],
       [ 79],
       [127],
       [ 65],
       [ 97],
       [  0],
       [101],
       [ 34],
       [ 98],
       [146],
       [ 95],
       [ 28],
       [ 23],
       [ 65],
       [  1],
       [  0],
       [  9],
       [  0],
       [  8],
       [114],
       [ 44],
       [102],
       [154],
       [ 17],
       [127],
       [142],
       [ 77],
       [ 67],
       [ 14],
       [ 65],
       [106],
       [115],
       [ 71],
       [157],
       [ 88],
       [ 10],
       [ 98],
       [111],
       [ 32],
       [ 28],
       [101],
       [ 84],
       [ 98],
       [  4],
       [  0],
       [ 85],
       [ 89],
       [ 66],
       [ 73],
       [108],
       [ 40],
       [153],
       [145],
       [158],
       [144],
       [113],
       [ 11],
       [ 38],
       [108],
       [163],
       [154],
       [ 86],
      

In [17]:
Y_train = to_categorical(Y_train, vocab_size)
Y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# Train Model

In [18]:
cbow = Sequential()

In [19]:
cbow.add(Embedding(input_dim=vocab_size, output_dim=dim, input_length=c*2))

In [20]:
cbow.add(Lambda(lambda x : K.mean(x, axis=1), output_shape=(dim,)))

In [21]:
cbow.add(Dense(vocab_size, activation="softmax"))

In [22]:
cbow.compile(optimizer=SGD(), loss="categorical_crossentropy", metrics=["accuracy"])

In [23]:
cbow.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 100)            16500     
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 165)               16665     
                                                                 
Total params: 33165 (129.55 KB)
Trainable params: 33165 (129.55 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [24]:
cbow.fit(X_train,Y_train,epochs=5000)

Epoch 1/5000
9/9 [==============================] - 0s 3ms/step - loss: 5.1058 - accuracy: 0.0000e+00
Epoch 2/5000
9/9 [==============================] - 0s 4ms/step - loss: 5.1051 - accuracy: 0.0000e+00
Epoch 3/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.1043 - accuracy: 0.0037
Epoch 4/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.1035 - accuracy: 0.0073
Epoch 5/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.1028 - accuracy: 0.0147
Epoch 6/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.1020 - accuracy: 0.0256
Epoch 7/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.1012 - accuracy: 0.0366
Epoch 8/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.1005 - accuracy: 0.0476
Epoch 9/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0997 - accuracy: 0.0513
Epoch 10/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0990 - accuracy: 0.0549
Epoch 11/50

9/9 [==============================] - 0s 2ms/step - loss: 5.0466 - accuracy: 0.0586
Epoch 84/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0459 - accuracy: 0.0586
Epoch 85/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0452 - accuracy: 0.0586
Epoch 86/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0445 - accuracy: 0.0586
Epoch 87/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0439 - accuracy: 0.0586
Epoch 88/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0432 - accuracy: 0.0586
Epoch 89/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0425 - accuracy: 0.0586
Epoch 90/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0418 - accuracy: 0.0586
Epoch 91/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0411 - accuracy: 0.0586
Epoch 92/5000
9/9 [==============================] - 0s 2ms/step - loss: 5.0405 - accuracy: 0.0586
Epoch 93/5000
9/9 [=====

9/9 [==============================] - 0s 2ms/step - loss: 4.9932 - accuracy: 0.0586
Epoch 166/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9926 - accuracy: 0.0586
Epoch 167/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9920 - accuracy: 0.0586
Epoch 168/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9913 - accuracy: 0.0586
Epoch 169/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9908 - accuracy: 0.0586
Epoch 170/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9901 - accuracy: 0.0586
Epoch 171/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9895 - accuracy: 0.0586
Epoch 172/5000
9/9 [==============================] - 0s 3ms/step - loss: 4.9889 - accuracy: 0.0586
Epoch 173/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9883 - accuracy: 0.0586
Epoch 174/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9877 - accuracy: 0.0586
Epoch 175/5000


9/9 [==============================] - 0s 2ms/step - loss: 4.9463 - accuracy: 0.0586
Epoch 248/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9458 - accuracy: 0.0586
Epoch 249/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9453 - accuracy: 0.0586
Epoch 250/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9448 - accuracy: 0.0586
Epoch 251/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9442 - accuracy: 0.0586
Epoch 252/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9437 - accuracy: 0.0586
Epoch 253/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9432 - accuracy: 0.0586
Epoch 254/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9427 - accuracy: 0.0586
Epoch 255/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9422 - accuracy: 0.0586
Epoch 256/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9417 - accuracy: 0.0586
Epoch 257/5000


9/9 [==============================] - 0s 2ms/step - loss: 4.9063 - accuracy: 0.0586
Epoch 330/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9058 - accuracy: 0.0586
Epoch 331/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9054 - accuracy: 0.0586
Epoch 332/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9049 - accuracy: 0.0586
Epoch 333/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9045 - accuracy: 0.0586
Epoch 334/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9040 - accuracy: 0.0586
Epoch 335/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9036 - accuracy: 0.0586
Epoch 336/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9032 - accuracy: 0.0586
Epoch 337/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9027 - accuracy: 0.0586
Epoch 338/5000
9/9 [==============================] - 0s 2ms/step - loss: 4.9023 - accuracy: 0.0586
Epoch 339/5000


...

9/9 [==============================] - 0s 2ms/step - loss: 3.1660 - accuracy: 0.3150
Epoch 4631/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1652 - accuracy: 0.3150
Epoch 4632/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1645 - accuracy: 0.3150
Epoch 4633/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1637 - accuracy: 0.3150
Epoch 4634/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1630 - accuracy: 0.3150
Epoch 4635/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1622 - accuracy: 0.3150
Epoch 4636/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1615 - accuracy: 0.3150
Epoch 4637/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1607 - accuracy: 0.3150
Epoch 4638/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1599 - accuracy: 0.3150
Epoch 4639/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1592 - accuracy: 0.3150
Epoch 

9/9 [==============================] - 0s 2ms/step - loss: 3.1047 - accuracy: 0.3187
Epoch 4712/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1039 - accuracy: 0.3187
Epoch 4713/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1032 - accuracy: 0.3187
Epoch 4714/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1024 - accuracy: 0.3187
Epoch 4715/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1017 - accuracy: 0.3187
Epoch 4716/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1009 - accuracy: 0.3187
Epoch 4717/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.1002 - accuracy: 0.3187
Epoch 4718/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0994 - accuracy: 0.3187
Epoch 4719/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0986 - accuracy: 0.3187
Epoch 4720/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0979 - accuracy: 0.3187
Epoch 

9/9 [==============================] - 0s 2ms/step - loss: 3.0432 - accuracy: 0.3260
Epoch 4793/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0425 - accuracy: 0.3260
Epoch 4794/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0417 - accuracy: 0.3260
Epoch 4795/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0410 - accuracy: 0.3260
Epoch 4796/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0402 - accuracy: 0.3260
Epoch 4797/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0395 - accuracy: 0.3260
Epoch 4798/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0387 - accuracy: 0.3260
Epoch 4799/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0379 - accuracy: 0.3260
Epoch 4800/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0372 - accuracy: 0.3260
Epoch 4801/5000
9/9 [==============================] - 0s 2ms/step - loss: 3.0364 - accuracy: 0.3297
Epoch 

9/9 [==============================] - 0s 2ms/step - loss: 2.9818 - accuracy: 0.3407
Epoch 4874/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9810 - accuracy: 0.3443
Epoch 4875/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9802 - accuracy: 0.3443
Epoch 4876/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9795 - accuracy: 0.3443
Epoch 4877/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9787 - accuracy: 0.3443
Epoch 4878/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9780 - accuracy: 0.3443
Epoch 4879/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9772 - accuracy: 0.3443
Epoch 4880/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9765 - accuracy: 0.3443
Epoch 4881/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9757 - accuracy: 0.3443
Epoch 4882/5000
9/9 [==============================] - 0s 3ms/step - loss: 2.9749 - accuracy: 0.3443
Epoch 

9/9 [==============================] - 0s 2ms/step - loss: 2.9203 - accuracy: 0.3516
Epoch 4955/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9195 - accuracy: 0.3516
Epoch 4956/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9188 - accuracy: 0.3516
Epoch 4957/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9180 - accuracy: 0.3516
Epoch 4958/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9172 - accuracy: 0.3516
Epoch 4959/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9165 - accuracy: 0.3516
Epoch 4960/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9157 - accuracy: 0.3516
Epoch 4961/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9149 - accuracy: 0.3516
Epoch 4962/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9142 - accuracy: 0.3516
Epoch 4963/5000
9/9 [==============================] - 0s 2ms/step - loss: 2.9134 - accuracy: 0.3516
Epoch 

In [25]:
pred = cbow.predict(X_train)

9/9 [==============================] - 0s 2ms/step


In [26]:
pred = pred.argmax(axis=1)

In [27]:
pred

array([108, 108,  44,  44,  28,   0,  65,   0,   0,  98,   0,   0,  98,
        98,   0,   0,  34,  98,   0, 101,  28,   0,  65,   0,   0,  65,
         0,   8, 114,  44,   8, 142,  17, 127, 142, 142, 127,   0,  65,
         0,   0,  98,  98,   0,  55,  98,   0,   0,  98, 101,   0,  98,
         0,   0,  98,  98,  66, 153, 108, 108, 153, 108, 108, 127, 108,
       153, 108, 108,   8, 154, 108,   8,   8, 108,   8,   8, 108,  44,
         8, 108, 108, 153, 108, 108, 108, 108, 108, 113,  98, 127, 113,
        50,   0, 127,  98,   0, 127, 142, 154,   8,   8, 108, 114,   0,
         0,  65, 127,   0,  99,  75,  75,  75,  75,  75,  75,   0,  75,
         0,   0,  75,  75,  99, 142,  99,  99, 142, 142, 127,  98,   0,
       127,  98,  55,  55,  98,  98,  99, 142,  99,  99,  64, 142,  99,
        99,  99, 142,  99,  99, 142,  98,   0,   0,   0,  98,   0,  55,
        98, 108, 108, 108,   0,  98, 101,   0,  98, 101,   0,   0,  98,
        99,  99,  99,  99,  81,  34,  98,  56, 129,  98,  56,  5

In [28]:
i = 0
for context in X_train:
    print("==================Context================")
    for w in context:
        print(idx_to_words[w])
    print("=================Predicted================")
    print(idx_to_words[pred[i]])
    i+=1

==================Context================
when
zarathustra
thirty
years
=================Predicted================
thou
==================Context================
zarathustra
was
years
old
=================Predicted================
thou
==================Context================
was
thirty
old
he
=================Predicted================
years
==================Context================
thirty
years
he
left
=================Predicted================
years
==================Context================
years
old
left
his
=================Predicted================
he
==================Context================
old
he
his
home
=================Predicted================
and
==================Context================
he
left
home
and
=================Predicted================
his
==================Context================
left
his
and
the
=================Predicted================
and
==================Context================
his
home
the
lake
=================Predicted================
and
============